# Generated Summaries Quality Metrics

Three simple metrics to validate our 4,626 generated summaries are good quality.

In [ ]:
# Setup: Load data
import json
import pandas as pd
import numpy as np
import re

with open('generated_summaries.json', 'r', encoding='utf-8') as f:
    summaries = json.load(f)

df = pd.read_csv('cleaned_resumes.csv')
df['generated_summary'] = df.index.astype(str).map(summaries)

print(f"Loaded {len(summaries)} generated summaries")

## Metric 1: Word Count
Are the summaries a reasonable length? (Not too short, not too long)

In [ ]:
word_counts = df['generated_summary'].dropna().apply(lambda x: len(str(x).split()))

print(f"Average word count: {word_counts.mean():.0f} words")
print(f"Range: {word_counts.min()} to {word_counts.max()} words")
print(f"Summaries too short (<50 words): {(word_counts < 50).sum()}")

## Metric 2: Skill Coverage
Do the summaries mention skills from the original resume? (Content faithfulness)

In [ ]:
def get_skill_coverage(row):
    skills_str = row['skills']
    summary = str(row['generated_summary']).lower()
    if pd.isna(skills_str): return np.nan
    
    skills = re.findall(r":\s*([^(]+?)\s*\(Level:", str(skills_str))
    skills = [s.strip().lower() for s in skills if s.strip()]
    if not skills: return np.nan
    
    mentioned = sum(1 for skill in skills if skill in summary)
    return (mentioned / len(skills)) * 100

df['skill_coverage'] = df.apply(get_skill_coverage, axis=1)
coverage = df['skill_coverage'].dropna()

print(f"Average skill coverage: {coverage.mean():.1f}%")
print(f"Summaries with >30% coverage: {(coverage > 30).sum()} / {len(coverage)} ({(coverage > 30).sum()/len(coverage)*100:.1f}%)")

## Metric 3: Years Accuracy
When the summary mentions years of experience, is it accurate?

In [ ]:
def extract_years(text):
    match = re.search(r'(\d+(?:\.\d+)?)\s*(?:\+)?\s*years?', str(text).lower())
    return float(match.group(1)) if match else np.nan

df['original_years'] = df['total_experience_time'].str.replace(' Years', '').astype(float)
df['summary_years'] = df['generated_summary'].apply(extract_years)

valid = df.dropna(subset=['original_years', 'summary_years'])
valid['diff'] = abs(valid['summary_years'] - valid['original_years'])

print(f"Summaries where years could be compared: {len(valid)}")
print(f"Accurate within ±2 years: {(valid['diff'] <= 2).sum()} / {len(valid)} ({(valid['diff'] <= 2).sum()/len(valid)*100:.1f}%)")
print(f"Average difference: {valid['diff'].mean():.1f} years")